<a href="https://colab.research.google.com/github/fabiomatricardi/GradioLangchainCustomLLM/blob/main/Gradio%2BLangChain%3DFreeAI_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade --quiet  gradio_tools huggingface_hub langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
from gradio_client import Client

yourHFtoken = "hf_xxxxxxxxxxxxxxxxxxx" #here your HF token
client = Client("ysharma/Chat_with_Meta_llama3_8b", hf_token=yourHFtoken)
result = client.predict(
		message="Why you should learn AI?",
		request=0.95,
		param_3=512,
		api_name="/chat"
)
print(result)

Loaded as API: https://ysharma-chat-with-meta-llama3-8b.hf.space ✔
assistant

Learning AI can be incredibly valuable in today's rapidly evolving digital landscape. Here are some compelling reasons why you should consider learning AI:

1. **Job Opportunities**: The AI industry is growing rapidly, and there is a high demand for skilled AI professionals. According to Glassdoor, the average salary for an AI engineer is around $141,000 per year. Learning AI can open up new job opportunities and increase your earning potential.
2. **Improve Problem-Solving Skills**: AI involves solving complex problems using data, algorithms, and machine learning. Developing AI skills can improve your problem-solving abilities and enhance your critical thinking skills.
3. **Enhance Career Prospects**: As AI becomes increasingly integrated into various industries, companies are looking for professionals who can work with AI systems. Learning AI can enhance your career prospects and make you a more attractive 

### Create the New Class for a Custom LLM in Langchain

In [3]:
from typing import Any, Dict, Iterator, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

class GradioClientChat(LLM):
    """
    Custom LLM class based on the Gradio API call.
    """
    from gradio_client import Client
    chatbot: Any = None

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # Instantiating the ChatBot class
        # add here you hf_token, in case as shown here below
        #yourHFtoken = "hf_xxxxxxxxxxxxxxxxx" #here your HF token
        #self.chatbot =("ysharma/Chat_with_Meta_llama3_8b", hf_token=yourHFtoken)
        self.chatbot = Client("ysharma/Chat_with_Meta_llama3_8b")

    @property
    def _llm_type(self) -> str:
        return "Gradio API client Meta_llama3_8b"

    def _call(
            self,
            prompt: str,
            stop: Optional[List[str]] = None,
            run_manager: Optional[CallbackManagerForLLMRun] = None,
            chatbot=None,
            request: float = 0.95,
            param: float = 512,
    ) -> str:
        """
        Make an API call to the Gradio API client Meta_llama3_8b using the specified prompt and return the response.
        """
        if chatbot is None:
            chatbot = self.chatbot

        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        # Return the response from the API
        result = chatbot.predict(   #.submit for streaming effect / .predict for normal output
            		message=prompt,
                request=request,
                param_3=param,
                api_name="/chat"
        )
        return str(result)

    def _stream(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        chatbot=None,
        request: float = 0.95,
        param: float = 512,
        **kwargs: Any,
    ) -> Iterator[GenerationChunk]:
        """Stream the LLM on the given prompt.

        This method should be overridden by subclasses that support streaming.

        If not implemented, the default behavior of calls to stream will be to
        fallback to the non-streaming version of the model and return
        the output as a single chunk.

        Args:
            prompt: The prompt to generate from.
            stop: Stop words to use when generating. Model output is cut off at the
                first occurrence of any of these substrings.
            run_manager: Callback manager for the run.
            **kwargs: Arbitrary additional keyword arguments. These are usually passed
                to the model provider API call.

        Returns:
            An iterator of GenerationChunks.
        """
        if chatbot is None:
            chatbot = self.chatbot

        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        # Return the response from the API
        for char in chatbot.submit(   #.submit for streaming effect / .predict for normal output
            		message=prompt,
                request=request,
                param_3=param,
                api_name="/chat"
                ):
            chunk = GenerationChunk(text=char)
            if run_manager:
                run_manager.on_llm_new_token(chunk.text, chunk=chunk)

            yield chunk

> instantiate the Custom Langchain LLM

In [4]:
llm = GradioClientChat()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loaded as API: https://ysharma-chat-with-meta-llama3-8b.hf.space ✔


> run wihtout streaming

In [5]:
result = llm.invoke("what is artificial Intelligence?")  #[10:]   to remove the assitant from the output
print(result)

assistant

Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as:

1. **Learning**: AI systems can learn from data, adapt to new situations, and improve their performance over time.
2. **Reasoning**: AI systems can make decisions, solve problems, and draw conclusions based on data and rules.
3. **Perception**: AI systems can interpret and understand data from sensors, images, and audio.
4. **Problem-solving**: AI systems can identify and solve complex problems, often through trial and error.

AI is a broad field that encompasses many subfields, including:

1. **Machine Learning** (ML): A type of AI that enables systems to learn from data and improve their performance over time.
2. **Natural Language Processing** (NLP): A type of AI that enables computers to understand, interpret, and generate human language.
3. **Computer Vision**: A type of AI that enables computers to interpret and understa

#### change few parameters for test
- temperature = 0.45
- max new tokens 600
- remove the first 10 chars

In [7]:
result = llm.invoke("what are the differences between artificial Intelligence and machine learning?", request = 0.45, param = 600)[10:]  # to remove the assitant from the output
print(result)


Artificial Intelligence (AI) and Machine Learning (ML) are often used interchangeably, but they are not exactly the same thing. Here's a breakdown of the differences:

**Artificial Intelligence (AI)**

Artificial Intelligence refers to the broader field of research and development aimed at creating machines that can perform tasks that typically require human intelligence, such as:

1. **Reasoning**: AI systems can reason and make decisions based on data and rules.
2. **Problem-solving**: AI systems can solve complex problems, often through trial and error.
3. **Perception**: AI systems can perceive and understand data from sensors, images, and audio.
4. **Learning**: AI systems can learn from data and adapt to new situations.

AI is a multidisciplinary field that encompasses many subfields, including:

1. **Natural Language Processing** (NLP)
2. **Computer Vision**
3. **Robotics**
4. **Expert Systems**

**Machine Learning (ML)**

Machine Learning is a subset of Artificial Intelligence

> call the streaming effect method

In [8]:
final = ''
for token in llm.stream("what is the scientific method?",request = 0.25, param = 600):
        if final == '':
            final=token
            print(token, end="", flush=True)
        else:
            try:
                print(token.replace(final,''), end="", flush=True)
                final = token
            except:
                pass

assistant

The scientific method is a systematic process used to develop and test scientific knowledge. It involves making observations, formulating hypotheses, and testing those hypotheses through experimentation and data analysis. The scientific method is a fundamental part of the scientific process and is used in many fields, including physics, biology, chemistry, and more.

The steps of the scientific method are:

1. **Make an observation**: Scientists start by making an observation about the world around them. This can be something they notice in their everyday lives, or it can be a phenomenon that has been observed by others.
2. **Ask a question**: Based on their observation, scientists ask a question about what they have observed. This question should be specific and testable.
3. **Research the topic**: Scientists do some background research on the topic to learn more about what is already known about it. This helps them to understand the context of their observation and to iden

In [10]:
final = ''
for token in llm.stream("List 5 reasons why vegan food is healthy",request = 0.25, param = 600):
        if final == '':
            final=token
            print(token, end="", flush=True)
        else:
            try:
                print(token.replace(final,''), end="", flush=True)
                final = token
            except:
                pass

assistant

Here are 5 reasons why vegan food is considered healthy:

1. **High in Fiber**: Vegan diets tend to be high in fiber, which can help promote digestive health, prevent constipation, and support healthy blood sugar levels. Fiber-rich foods like fruits, vegetables, and whole grains can also help lower cholesterol levels and reduce the risk of heart disease.

2. **Rich in Antioxidants**: Vegan diets are rich in antioxidants, which are essential for protecting the body against free radicals and oxidative stress. Antioxidants can help reduce the risk of chronic diseases like cancer, heart disease, and neurodegenerative disorders. Fruits, vegetables, and nuts are all high in antioxidants.

3. **Low in Saturated Fat**: Vegan diets tend to be low in saturated fat, which can help reduce the risk of heart disease and stroke. Saturated fats can also increase cholesterol levels and contribute to weight gain. Vegan sources of fat, such as avocados and nuts, are generally healthier than an




---


---



---

